In [6]:
%reload_ext autoreload
%autoreload
import pickle, os, json
import torch
from pathlib import Path, PureWindowsPath, PurePosixPath
from util.tools import load_config
import pandas as pd
from sklearn.metrics import classification_report

In [7]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [8]:
config_folder = "config/"
config = load_config(config_folder)
config.__dict__

{'hyperparams': <util.tools.config at 0x22fa66f0f70>,
 'programsettings': <util.tools.config at 0x22fa66f06d0>,
 'modelconfig': <util.tools.config at 0x22fa66f0a90>}

In [10]:
reports_folder_name = config.programsettings.REPORTS_DIR
data_folder = Path(reports_folder_name)
list_results_files = list(data_folder.glob('**/multi_model_*.pkl'))

all_results = []
all_data_source = []

for results_file in list_results_files:
#     print(results_file)
    with open(results_file,"rb") as f:
        lst_results = pickle.load(f)
#     print("File Name:", results_file, ' \n results: ', lst_results)
    
    for results in lst_results:
        max_seq = 0
        lr = 0
        train_batch = 0
        max_seq = ""
        model_name = ""
        class_weights = '[]'
        kernel_1 = 0
        kernel_2 = 0
        kernel_3 = 0
        if (len(results[0]) > 50):
#             print(results[0])
            results_json = json.loads(results[0])
            max_seq = results_json["hyperparams"]["MAX_SEQ_LENGTH"]
            lr = results_json["hyperparams"]["LEARNING_RATE"]
            train_batch = results_json["hyperparams"]["TRAIN_BATCH_SIZE"]
            model_name = results_json["programsettings"]["MODEL_NAME"]
            try:
                class_weights = results_json["hyperparams"]["LOSS_FN_CLASS_WEIGHTS"]
            
            except: 
                class_weights = '[]'
                
            try:
                kernel_1 = results_json["modelconfig"]["KERNEL_1"]
                kernel_2 = results_json["modelconfig"]["KERNEL_2"]
                kernel_3 = results_json["modelconfig"]["KERNEL_3"]
            
            except: 
                class_weights = '[]'

        result_store = [results_file, model_name,lr, train_batch,max_seq, class_weights,kernel_1, kernel_2,kernel_3,results[1], \
                        results[2], results[3], results[4], results[5],results[6], results[15], results[16] ]
        all_results.append(result_store)
        
#       Creating list of all dev_labels and dev_preds, so that we can run classification reports for all of them together
        if (len(results) > 7):
            all_data_source.append([results[7],results[8], results[13], results[14], result_store])
                                
    
columns = ['file_name','model_name', 'lr', 'train_batch_size','train_max_seq','class_weights','Kernel_1','Kernel_2','Kernel_3','train_loss', \
           'dev_loss', 'train_mcc', 'train_f1_score','dev_mcc','dev_f1_score', 'test_mcc', 'test_f1_score' ]

df_results = pd.DataFrame(all_results, columns = columns)
pd.set_option('display.max_colwidth', 100)
df_results

,file_name,model_name,lr,train_batch_size,train_max_seq,class_weights,Kernel_1,Kernel_2,Kernel_3,train_loss,dev_loss,train_mcc,train_f1_score,dev_mcc,dev_f1_score,test_mcc,test_f1_score
0,reports\re\multi_model_experiment_results_2020-11-30 14_26_09_634176.pkl,BioBERT_CNN_fc,0.00001,24,256,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",2,3,4,2.184182,2.186153,-0.015727,0.073449,-0.014934,0.07267,-0.013602,0.045466


### Let us generate Classification report for all the things we have available

In [11]:
for i in range(len(all_data_source)):
    dev_labels, dev_preds, test_preds, test_labels, model_config = all_data_source[i]
    print("\n \n \n",model_config)
    print(classification_report(dev_labels,dev_preds ))
    print(classification_report(test_labels,test_preds ))
    


 
 
 [WindowsPath('reports/re/multi_model_experiment_results_2020-11-30 14_26_09_634176.pkl'), 'BioBERT_CNN_fc', 1e-05, 24, 256, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 2, 3, 4, 2.1841821670532227, 2.1861525535583497, -0.015727100503186858, 0.07344949675776744, -0.014933973451609156, 0.07267033478415591, -0.013602342968071335, 0.045465608182594694]
              precision    recall  f1-score   support

           0       0.04      0.06      0.05        18
           1       0.00      0.00      0.00         5
           2       0.12      0.30      0.17        10
           3       0.00      0.00      0.00         7
           4       0.17      0.33      0.22         3
           5       0.06      0.17      0.09         6
           6       0.00      0.00      0.00         5
           7       0.04      0.50      0.08         2
           8       1.00      0.02      0.05        43

    accuracy                           0.08        99
   macro avg       0.16      0.15      0.07  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
